In [59]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

import json
import pandas as pd
pd.set_option("display.max_columns", 30)

import requests

In [2]:
"""
1. get the data from S3
2. weather date transformations
3. taxi_trips transformations
4. update payment_type_master
5. update company_master
6. update taxi_trips with company and payment type ids (replace the string values with ids from latest master tables)
7. upload weather date to s3
8. upload taxi date to s3
9. upload the newest payment tyope master and company master date to s3 as well
"""

'\n1. get the data from S3\n2. weather date transformations\n3. taxi_trips transformations\n4. update payment_type_master\n5. update company_master\n6. update taxi_trips with company and payment type ids (replace the string values with ids from latest master tables)\n7. upload weather date to s3\n8. upload taxi date to s3\n9. upload the newest payment tyope master and company master date to s3 as well\n'

## taxi trips transformation codes

In [3]:
#extract T-2 month's Chicaho taxi trips data.

current_date_time = datetime.now() - relativedelta(months=2)
formated_datetime = current_date_time.strftime("%Y-%m-%d")

url = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp >= '{formated_datetime}T00:00:00' AND trip_start_timestamp <= '{formated_datetime}T23:59:59'&$limit=30000"

headers = {"X-App-Token" : os.environ.get("CHICAGO_API_TOKEN")}

response = requests.get(url)

data = response.json()

In [4]:
taxi_trips = pd.DataFrame(data)
taxi_trips.sample(1)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
9210,1e80cfe0737b6522d222ea5dcc81aa7c6b425571,19bdccbc936f2f02882b69fd9e71ad97b9e783ba96a8c3...,2024-10-01T14:15:00.000,2024-10-01T15:00:00.000,2789,18.29,41,2,52.76,0,0,0,52.76,Cash,Flash Cab,41.794090253,-87.592310855,"{'type': 'Point', 'coordinates': [-87.59231085...",42.001571027,-87.695012589,"{'type': 'Point', 'coordinates': [-87.69501258...",NaN,NaN


In [5]:
# sensus columns do not contain real data, so deleted 
taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract"], axis=1, inplace=True)
taxi_trips.drop(["dropoff_centroid_location", "pickup_centroid_location"], axis=1, inplace=True)

# drop the nan values
taxi_trips.dropna(inplace=True)

#rename columns
taxi_trips.rename(columns={"pickup_community_area" : "pickup_community_area_id",
                          "dropoff_community_area" : "dropoff_community_area_id"}, inplace=True)

#create the weather helper column
taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("h")
# rounding to hour
#taxi_trips["datetime_for_weather"] = taxi_trips["datetime_for_weather"].dt.floor("h")

In [6]:
taxi_trips.sample(3)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
15927,b4fb99ff58ca33dcac12cc0a34966ec23d6bc7e9,cf9d594763b7b42b84d0a3c61f7ce63e7f491b3b0bb6a2...,2024-10-01T08:45:00.000,2024-10-01T09:45:00.000,3231,25.38,76,32,62,10.5,0,7.5,80.5,Mobile,5 Star Taxi,41.97907082,-87.903039661,41.880994471,-87.632746489,2024-10-01 08:00:00
605,77bf8664b2dde0406d7fd8789b589e16f7141295,30a2ec0ae79a56492f2b7301c8084c0e7bc56b2c84b1c6...,2024-10-01T22:00:00.000,2024-10-01T22:30:00.000,1800,18.1,76,8,45.25,10.15,0,5,60.4,Credit Card,Taxi Affiliation Services,41.980264315,-87.913624596,41.899602111,-87.633308037,2024-10-01 22:00:00
15726,4a16d5f4006f9dc27db48d64cbdc20a0a09c82f7,b4918e3dc616260ba3c7c57ccf5345c43f853abd067d61...,2024-10-01T09:00:00.000,2024-10-01T09:15:00.000,1380,3.8,43,41,15,3.3,0,1,19.3,Credit Card,Taxi Affiliation Services,41.761577908,-87.572781987,41.794090253,-87.592310855,2024-10-01 09:00:00


#### taxi trips tranformation function

In [7]:
def taxi_trips_transformation(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    """Performes transformation with the taxi data.

    Parameters
    ----------
    taxi trips: pd.DataFrame
        Tha DataFrame holding the daily taxi trips

    Returns
    -------
    pd.DataFrame
        The cleand transformd DataFrame holdiing the daily taxi trips 

    """

    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError("taxi trips is not a valid pandas DateFrame.")
    
    taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract", "dropoff_centroid_location",
                      "pickup_centroid_location"], axis=1, inplace=True)

    taxi_trips.dropna(inplace=True)

    taxi_trips.rename(columns={"pickup_community_area" : "pickup_community_area_id",
                            "dropoff_community_area" : "dropoff_community_area_id"}, inplace=True)

    taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("h")

    return taxi_trips

    

#### Company code updates

In [8]:
company_master = taxi_trips["company"].drop_duplicates().reset_index(drop=True)

company_master = pd.DataFrame(
    {
    "company_id" : range(1, len(company_master)+1),
    "company" : company_master
    }
)
company_master.tail(2)

,company_id,company
31,32,Metro Jet Taxi A.
32,33,Tac - Yellow Non Color


In [9]:
new_company_data =[
    {"company" : "Tac - Blue Diamond Dispatch"},
    {"company" : "X"},
    {"company" :  "Y"}
]

new_company_mapping = pd.DataFrame(new_company_data)
new_company_mapping

,company
0,Tac - Blue Diamond Dispatch
1,X
2,Y


In [10]:
company_max_id = company_master["company_id"].max()
company_max_id

33

In [11]:
new_company_list = []

for company in new_company_mapping["company"].values:
    if company not in company_master["company"].values:
        new_company_list.append(company)

#one line
new_company_list_one_line= [company for company in new_company_mapping["company"].values if company not in company_master["company"].values]

new_company_list_one_line

['X', 'Y']

In [12]:
new_companies_df = pd.DataFrame({
    "company_id": range(company_max_id + 1, company_max_id + len(new_company_list)+1),
    "company" : new_company_list
    })
new_companies_df

,company_id,company
0,34,X
1,35,Y


In [13]:
updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)
updated_company_master.tail(5)

,company_id,company
30,31,Star North Taxi Management Llc
31,32,Metro Jet Taxi A.
32,33,Tac - Yellow Non Color
33,34,X
34,35,Y


In [14]:
def updated_company_master(taxi_trips: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    """Ecxtend the the company master with the new companies if there are new companies
   
    Parameters
    ----------
    taxi trips: pd.DataFrame
        DateFrame holding the daily taxi trips
    company_master: pd.DataFrame
        DateFrame holding the company masters data
    Returns
    -------
    pd.DataFrame
        the updated company masters data, if new companies are in the taxi data, they will be loaded to it
        
    
    
    """
    company_max_id = company_master["company_id"].max()
   
    new_company_list= [company for company in taxi_trips["company"].values if company not in company_master["company"].values]

    new_companies_df = pd.DataFrame({
    "company_id": range(company_max_id + 1, company_max_id + len(new_company_list)+1),
    "company" : new_company_list
    })

    updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

    return updated_company_master


In [15]:
taxi_trips_company_only = pd.DataFrame({
    "company_id" : [1, 2, 3],
    "company" : ["Tac - Yellow Non Color", "x","y"]
})
taxi_trips_company_only 

,company_id,company
0,1,Tac - Yellow Non Color
1,2,x
2,3,y


In [16]:
updated_company_master = updated_company_master(taxi_trips=taxi_trips_company_only, company_master=company_master)


In [17]:
updated_company_master.tail(3)

,company_id,company
32,33,Tac - Yellow Non Color
33,34,x
34,35,y


### payment type master code

In [25]:
payment_type_master = taxi_trips["payment_type"].drop_duplicates().reset_index(drop=True)

payment_type_master = pd.DataFrame(
    {
    "payment_type_id" : range(1, len(payment_type_master)+1),
    "payment_type" : payment_type_master
    }
)

taxi_trips_payment_type_only = pd.DataFrame({
    "payment_type_id" : [1, 2, 3],
    "payment_type" : ["Cash", "x","y"]
})

In [26]:
taxi_trips_payment_type_only 

,payment_type_id,payment_type
0,1,Cash
1,2,x
2,3,y


In [27]:
def updated_payment_type_master(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame) -> pd.DataFrame:
    """Ecxtend the the payment_type master with the new payment_type if there are new payment_types
   
    Parameters
    ----------
    taxi trips: pd.DataFrame
        DateFrame holding the daily taxi trips
    payment_type_master: pd.DataFrame
        DateFrame holding the payment_type masters data
    Returns
    -------
    pd.DataFrame
        the updated payment_type masters data, if new companies are in the taxi data, they will be loaded to it
        
    
    
    """
    payment_type_max_id = payment_type_master["payment_type_id"].max()
   
    new_payment_types_list= [payment_type for payment_type in taxi_trips["payment_type"].values if payment_type not in payment_type_master["payment_type"].values]

    new_payment_types_df = pd.DataFrame({
    "payment_type_id": range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_types_list)+1),
    "payment_type" : new_payment_types_list
    })

    updated_payment_type_master = pd.concat([payment_type_master, new_payment_types_df], ignore_index=True)

    return updated_payment_type_master

In [28]:
updated_payment_type_master = updated_payment_type_master(taxi_trips=taxi_trips_payment_type_only, payment_type_master=payment_type_master)

In [31]:
updated_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Cash
2,3,Unknown
3,4,Mobile
4,5,Prcard
5,6,No Charge
6,7,Dispute
7,8,x
8,9,y


### creating a generic update master table function 

In [ ]:
def updated_master(taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str, value_column: str) -> pd.DataFrame:
    """Ecxtend the master DateFrame with the new values if there are new any
   
    Parameters
    ----------
    taxi trips: pd.DataFrame
        DateFrame holding the daily taxi trips
    master: pd.DataFrame
        DateFrame holding the masters data
    id_column: str
        The id column of the master DataFrame
    value_column
        Name of the column in master_df containing the values

    Returns
    -------
    pd.DataFrame
        the updated  masters data, if new values are in the taxi data, they will be loaded to it
        
    
    
    """
    max_id = master[id_column].max()
   
    new_values_list= [value for value in taxi_trips[value_column].values if value not in master[value_column].values]

    value_column_df = pd.DataFrame({
    id_column: range(max_id + 1, max_id + len(new_values_list)+1),
    value_column : new_values_list
    })

    updated_master = pd.concat([master, value_column_df ], ignore_index=True)

    return updated_master

In [37]:
test_payment_type_master = updated_master(taxi_trips=taxi_trips_payment_type_only, master=payment_type_master,
                                          id_column="payment_type_id", value_column="payment_type")

In [38]:
test_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Cash
2,3,Unknown
3,4,Mobile
4,5,Prcard
5,6,No Charge
6,7,Dispute
7,8,x
8,9,y


In [40]:
test_company_master = updated_master(taxi_trips=taxi_trips_company_only, master=company_master,
                                          id_column="company_id", value_column="company")

In [42]:
test_company_master.tail(5)

,company_id,company
30,31,Star North Taxi Management Llc
31,32,Metro Jet Taxi A.
32,33,Tac - Yellow Non Color
33,34,x
34,35,y


### update taxi trips with the most recent company master and payment type function

In [43]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,c93ebddf36273457b0156092784f88647f78f8f3,42560393a9c9b9ae28339f4b5aec77fd89bd49916ad541...,2024-10-01T23:45:00.000,2024-10-02T00:00:00.000,1198,9.43,32,19,25.25,6.44,0,0,32.19,Credit Card,Patriot Taxi Dba Peace Taxi Associat,41.878865584,-87.625192142,41.927260956,-87.765501609,2024-10-01 23:00:00
1,c8e87e513da66b687ce0a2b3258a5f29b64f8415,8d75a4e15682364840d85e0f9f954e98b2703f4e252e88...,2024-10-01T23:45:00.000,2024-10-02T00:15:00.000,1260,12.1,56,39,31.25,0,0,4,35.25,Cash,Taxi Affiliation Services,41.79259236,-87.769615453,41.808916283,-87.596183344,2024-10-01 23:00:00
2,c39f11fe5bdcc901be1e38c2c517799615dbf896,342474dd3a24c9d5b7dbc2514032a01ecb396624638be5...,2024-10-01T23:45:00.000,2024-10-02T00:30:00.000,2700,14.5,17,28,42.25,0,0,0,42.25,Cash,Taxi Affiliation Services,41.94651142,-87.806020002,41.874005383,-87.66351755,2024-10-01 23:00:00
3,beaad7be40b1d13a475be22dd0ce856c35aa8da8,ad0ad3b00e143317e02602396f16dde8fd30faefd491d6...,2024-10-01T23:45:00.000,2024-10-01T23:45:00.000,22,0.06,28,28,120,0,0,0,121,Credit Card,City Service,41.874005383,-87.66351755,41.874005383,-87.66351755,2024-10-01 23:00:00
4,be4f3dfdab5458800bebb604efeb316196c20885,2981edd199f55bdc8b5ed188de69e9f5d8116ae8d7a434...,2024-10-01T23:45:00.000,2024-10-02T00:00:00.000,720,12.1,53,35,30.25,0,0,0,30.25,Unknown,Taxi Affiliation Services,41.673819904,-87.635739777,41.835117986,-87.618677767,2024-10-01 23:00:00


In [53]:
def update_taxi_trips_with_master_data(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame, company_master: pd.DataFrame) ->pd.DataFrame:
    """Update the taxi trips DataFrame with company master ID and payment_type master ID and delete the string columns

    Parameters
    ----------
    taxi_trips : pd.DataFrame
        The DataFrame with the daily taxi trips,
    payment_type_master : pd.DataFrame
        The payment type master table
    company_master : pd.DataFrame
        The company type master table

    Returns
    -------
    pd.DataFrame
        The taxi trips data, with only payment type id and company id, without company or payment type values
    """
    
    taxi_trips_id = taxi_trips.merge(payment_type_master, on="payment_type")
    taxi_trips_id = taxi_trips_id.merge(company_master, on="company")
    
    taxi_trips_id.drop(["payment_type", "company"] ,axis=1, inplace=True)

    return taxi_trips_id 

In [56]:
taxi_trips_id = update_taxi_trips_with_master_data(taxi_trips=taxi_trips, payment_type_master= payment_type_master, company_master= company_master)

In [57]:
taxi_trips_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
9130,55c20aaa187ea6c489b44441e4478c8eb8fc7fee,3835dded237bfc6b12d2181ec37148bd8385590f81597f...,2024-10-01T13:15:00.000,2024-10-01T13:45:00.000,2165,17.64,8,76,44.5,0,0,0,44.5,41.899602111,-87.633308037,41.980264315,-87.913624596,2024-10-01 13:00:00,2,6
7818,0edc26efc257b25272ee0bf401f47064d14c3a68,841f99fdfdf5cedb4f5fc9a7b5e543b003bf736e7bbe14...,2024-10-01T14:30:00.000,2024-10-01T14:45:00.000,741,0,7,32,14.36,3.53,0,0,17.89,41.922686284,-87.649488729,41.878865584,-87.625192142,2024-10-01 14:00:00,4,7
11907,6ecf7a7d76ad07bd6e6db45eab6ae27736e5eaeb,4027b7b9d535491bc9eb8deb02f8aedfe5ff99f9712882...,2024-10-01T10:45:00.000,2024-10-01T10:45:00.000,46,0.03,76,76,3.25,0,0,0,3.25,41.97907082,-87.903039661,41.97907082,-87.903039661,2024-10-01 10:00:00,2,7
16260,22c416ad03bf33a69632c04813aac1097c4d1e44,bbdec1f9949f4e13be32d6d0038763a29f09ef18534115...,2024-10-01T05:45:00.000,2024-10-01T06:15:00.000,1506,12.27,4,76,31.75,6.98,0,0,38.73,41.975170943,-87.687515515,41.980264315,-87.913624596,2024-10-01 05:00:00,4,13
3481,b3693402722e0532387ed7bad01558cf6b23badf,eaf9ff311ef5bc5e9f673e209710280ab0cc565517ee62...,2024-10-01T18:00:00.000,2024-10-01T18:15:00.000,960,1.8,8,7,10.5,2.2,0,0,12.7,41.89503345,-87.619710672,41.914616286,-87.631717366,2024-10-01 18:00:00,1,2


### Weather transformation function 

In [60]:
def transform_weather_data(weather_data: json) ->pd.DataFrame:
    """Make transformation on the daily weather api response.

    Parameters
    ----------
    weather_data : json
         daily weather data from the Open Meteo API.

    Returns
    -------
    pd.DataFrame
        A DateFrame representation of the data
    """
    weather_data_filtered = {
        "datetime" : weather_data["hourly"]["time"],
        "temperature" : weather_data["hourly"]["temperature_2m"],
        "wind_speed" : weather_data["hourly"]["wind_speed_10m"],
        "rain" : weather_data["hourly"]["rain"],
        "precipitation" : weather_data["hourly"]["precipitation"]
    }

    weather_df = pd.DataFrame(weather_data_filtered)

    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"])
    
    return weather_df

In [63]:
# test
#extract part:

current_date_time = datetime.now() - relativedelta(months=2)
formated_datetime = current_date_time.strftime("%Y-%m-%d")

url = "https://archive-api.open-meteo.com/v1/era5"


params = {
    "latitude" : 41.85,
    "longitude" : -87.65,
    "start_date" : formated_datetime,
    "end_date" : formated_datetime,
    "hourly" : "temperature_2m,wind_speed_10m,rain,precipitation"
}
response = requests.get(url, params=params)

weather_data= response.json()

In [64]:
weather_data_df= transform_weather_data(weather_data)

In [66]:
weather_data_df.head()

,datetime,temperature,wind_speed,rain,precipitation
0,2024-10-01 00:00:00,20.5,6.0,0.0,0.0
1,2024-10-01 01:00:00,20.0,8.9,0.0,0.0
2,2024-10-01 02:00:00,19.7,9.3,0.0,0.0
3,2024-10-01 03:00:00,18.9,8.0,0.0,0.0
4,2024-10-01 04:00:00,18.5,8.3,0.0,0.0
